In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
import scipy
import dateutil
from sklearn.model_selection import train_test_split,KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from functools import reduce

In [2]:
csv_paths = ["features.csv", "binary_features_users.csv", "feature_hist.csv",
            #"number_sessions.csv", "accepted_rate_session.csv", "usage_duration_mean.csv"
            ]
df_paths_lists = ["data/train_data/" + k for k in csv_paths]
df_paths_test = ["data/test_data/" + k for k in csv_paths]

df_list_train = [pd.read_csv(path) for path in df_paths_lists]
df_list_test = [pd.read_csv(path) for path in df_paths_test]
df_train = reduce(lambda df1, df2: df1.merge(df2, "outer"), df_list_train)
df_test = reduce(lambda df1, df2: df1.merge(df2, "outer"), df_list_test)
# df_train = df_train.drop("Unnamed: 0", axis=1)
# df_test = df_test.drop("Unnamed: 0", axis=1)
df_test.fillna(0)
df_train.dropna(inplace=True)

features_names = list(df_train)
features_names.remove("id")
features_names.remove("churned")
features = df_train[features_names]
labels = df_train["churned"]
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2)

In [3]:
features_names

['num_usages',
 'is_australian',
 'is_app_version_196',
 'is_app_version_18',
 'is_ios_version_113',
 'is_ios_version_1126',
 'is_ios_version_1121',
 'is_ios_version_1125',
 'is_ios_version_1112',
 'is_ios_version_1033',
 'is_ios_version_1032',
 'is_ios_version_1122',
 'is_ios_version_1021',
 'is_ios_version_1103',
 'is_iphone7p',
 'is_iphoneX',
 'is_iphone8p',
 'is_iphone8',
 'is_iphone6',
 'is_iphone6p',
 'is_installed_2016',
 'is_installed_2017',
 'is_installed_2018',
 'is_installation_subscripiton_same_month',
 'Adjust',
 'Backdrop',
 'Crop',
 'Darkroom',
 'Defocus',
 'Eyes',
 'Face',
 'Filters',
 'Light FX',
 'Paint',
 'Patch',
 'Prism',
 'Relight',
 'Reshape',
 'Retouch',
 'Touch Up',
 'Vignette',
 'accepted']

In [4]:
len(features_names)

42

In [5]:
# # # features_names = ["example_feature","example_numeric"]
# # features_names = "num_usages"
# features = df[features_names] # can add more features
# labels = df["churned"]
# X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2)
# X_train = X_train.values#.reshape(-1,1)
# X_val = X_val.values#.reshape(-1,1)
# y_train = y_train.values
# y_val = y_val.values

In [6]:
features

,num_usages,is_australian,is_app_version_196,is_app_version_18,is_ios_version_113,is_ios_version_1126,is_ios_version_1121,is_ios_version_1125,is_ios_version_1112,is_ios_version_1033,...,Light FX,Paint,Patch,Prism,Relight,Reshape,Retouch,Touch Up,Vignette,accepted
0,39.0,False,False,False,False,False,False,True,False,False,...,15.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,12.0,0.871795
1,38.0,False,False,False,False,False,True,False,False,False,...,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,4.0,0.763158
2,6.0,False,False,False,False,False,False,False,False,True,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.333333
3,12.0,False,False,False,False,True,False,False,False,False,...,5.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.666667
4,4.0,False,False,False,False,True,False,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.750000
5,15.0,False,False,False,False,False,True,False,False,False,...,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.666667
6,22.0,False,False,False,False,True,False,False,False,False,...,5.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,8.0,0.363636
7,18.0,False,False,False,False,False,False,True,False,False,...,3.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,3.0,0.368421
8,17.0,False,False,False,False,False,True,False,False,False,...,2.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.294118
9,31.0,False,False,False,False,False,False,False,False,False,...,6.0,8.0,0.0,1.0,0.0,0.0,2.0,0.0,4.0,0.580645


In [22]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 20, num = 11)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [23]:
from sklearn.ensemble import RandomForestClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune


rf = RandomForestClassifier()
# Random search of parametrs, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = "f1")
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.4min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [3, 4, 6, 8, 9, 11, 13, 14, 16, 18, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='f1', verbose=2)

In [24]:
rf_random.best_params_


{'n_estimators': 2000,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 3,
 'bootstrap': True}

In [25]:
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [3, 4, 6, 8, 9, 11, 13, 14, 16, 18, 20],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [26]:
import sklearn
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    f1 = sklearn.metrics.f1_score(test_labels,predictions)
    return f1
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_f1_train = evaluate(base_model, X_train, y_train)
print(base_f1_train)
base_f1 = evaluate(base_model, X_val, y_val)
print(base_f1)

0.8873917228103947
0.0


In [27]:
best_random = rf_random.best_estimator_
best_f1_train = evaluate(best_random, X_train, y_train)
print(best_f1_train)
best_f1_val = evaluate(best_random, X_val, y_val)
print(best_f1_val)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0
0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
best_f = evaluate(best_random, X_val, y_val)
print(best_f1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


NameError: name 'best_f1' is not defined

In [ ]:
test_df = pd.read_csv(r"LightricksSource\test_users_data.csv")
# test_df = test_df.set_index(test_df["id"])
# test_df["example_feature"] = test_df["country"] == "United States"
# test_df["example_numeric"] = np.random.randint(1, 6, test_df.shape[0])
X_test = test_df[features_names]
y_test = best_random.predict(X_test)
test_df["churned"] = y_test
test_df = test_df.rename(index=str, columns={"Unnamed: 0": ""})
test_df.to_csv("submission.csv",index=False, quoting=1)

In [ ]:
test_df